In [3]:
#Nessesary libs
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet
from tensorflow.keras.applications import EfficientNetV2B0

target_shape = (200, 200)

In [3]:
cache_dir = Path(Path.home()) / ".keras"
anchor_images_path = cache_dir / "left"
positive_images_path = cache_dir / "right"
cache_dir

PosixPath('/home/bartlomiej/.keras')

In [4]:
!pip install --upgrade --no-cache-dir gdown
!gdown 1jvkbTr_giSP3Ru8OwGNCg6B4PvVbcO34
!gdown 1EzBZUb_mh_Dp_FKD0P4XiYYSd0QBH5zW
!unzip -oq left.zip -d $cache_dir
!unzip -oq right.zip -d $cache_dir

Downloading...
From (uriginal): https://drive.google.com/uc?id=1jvkbTr_giSP3Ru8OwGNCg6B4PvVbcO34
From (redirected): https://drive.google.com/uc?id=1jvkbTr_giSP3Ru8OwGNCg6B4PvVbcO34&confirm=t&uuid=449e5264-8af4-4fc2-bef3-6abecfd759b0
To: /home/bartlomiej/PycharmProjects/pythonProject/left.zip
100%|████████████████████████████████████████| 104M/104M [00:18<00:00, 5.48MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1EzBZUb_mh_Dp_FKD0P4XiYYSd0QBH5zW
From (redirected): https://drive.google.com/uc?id=1EzBZUb_mh_Dp_FKD0P4XiYYSd0QBH5zW&confirm=t&uuid=f1b75035-0c1b-41ce-b303-1e063303e4b6
To: /home/bartlomiej/PycharmProjects/pythonProject/right.zip
100%|████████████████████████████████████████| 104M/104M [00:09<00:00, 11.0MB/s]


In [5]:
def preprocess_image(filename):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    return image


def preprocess_pair(pic1, pic2):
    return (
        preprocess_image(pic1),
        preprocess_image(pic2),
    )

In [6]:
anchor_images = sorted(
    [str(anchor_images_path / f) for f in os.listdir(anchor_images_path)]
)

positive_images = sorted(
    [str(positive_images_path / f) for f in os.listdir(positive_images_path)]
)

image_count = len(anchor_images)


anchors = anchor_images*3
negative_images = anchor_images + positive_images
np.random.RandomState(seed=32).shuffle(negative_images)
labels  = tf.constant([1]*6016 + [0]*12032)

comps = positive_images + negative_images

#input_imgs = tf.stack([img_1, img_2], axis=0)
dataset = tf.data.Dataset.from_tensor_slices((anchors, positive_images))


2023-05-30 13:47:26.333489: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:47:26.358539: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:47:26.358669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:47:26.360066: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

ValueError: Dimensions 18048 and 6016 are not compatible

# Architektura

In [ ]:
from keras.layers import Dense, Concatenate, Reshape, Input, Conv2D, ReLU, BatchNormalization, GlobalAveragePooling2D
from keras.applications import EfficientNetV2B0

base_cnn = EfficientNetV2B0(
    weights="imagenet", input_shape=target_shape + (3,), include_top=False
)
flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(256)(dense2)

embedding = Model(base_cnn.input, output, name="Embedding")
embedding.summary()



img_input_1 = Input(target_shape, name="input_image_1")
img_input_2 = Input(target_shape, name="input_image_2")

feature_vector_A = embedding(img_input_1)
feature_vector_B = embedding(img_input_2)

joint_vector = Concatenate()([feature_vector_A, feature_vector_B])
edense1 = Dense(256*2, activation='relu', name='dense 1')(joint_vector)
edense2 = Dense(700, activation='relu', name='dense 3')(edense1)
edense3 = Dense(40, activation='relu', name='dense 3')(edense2)
eoutput = Dense(1, activation='sigmoid', name='output')(edense3)

model = Model(inputs = [img_input_1, img_input_2], outputs = output)
model.summary()

Model: "Embedding"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_7 (Rescaling)        (None, 200, 200, 3)  0           ['input_8[0][0]']                
                                                                                                  
 normalization_7 (Normalization  (None, 200, 200, 3)  0          ['rescaling_7[0][0]']            
 )                                                                                                
                                                                                          

ValueError: Exception encountered when calling layer 'normalization_7' (type Normalization).

Dimensions must be equal, but are 200 and 3 for '{{node Embedding/normalization_7/sub}} = Sub[T=DT_FLOAT](Embedding/rescaling_7/add, Embedding/normalization_7/sub/y)' with input shapes: [?,200,200], [1,1,1,3].

Call arguments received by layer 'normalization_7' (type Normalization):
  • inputs=tf.Tensor(shape=(None, 200, 200), dtype=float32)